<a href="https://colab.research.google.com/github/MarioSigal/Aprendizaje-Automatico-I-y-II/blob/main/TP2/tp2_Crear_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Imports


In [ ]:
!pip install ebooklib beautifulsoup4 pandas

In [ ]:
!pip install pdfplumber


In [ ]:
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import pandas as pd
import re
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pdfplumber
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Funciones Auxiliares

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

In [ ]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = str(t) if t is not None else ""
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def normalize_text_y(t):
    # Convertir a minúsculas y quitar puntuación
    t = str(t) if t is not None else ""
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-zA-Záéíóúüñ0-9¿?,.' -ÁÉÍÓÚÜÑ]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def transformar_etiqueta(y, indice):
  puntuacion_iniciales = []
  puntuacion_finales = []
  capitalizaciones = []
  instancia_ids = []
  token_ids = []
  tokens_l = []

  for parrafo in y:
    inicio_pregunta = False
    palabras = parrafo.split()


    for palabra in palabras:
      tokens = tokenizer.tokenize(palabra.lower())

      for i in range(len(tokens)):
        if tokens[i] == "¿":
          inicio_pregunta = True
          continue
        if tokens[i] == "?" or tokens[i] == "." or tokens[i] == ",":
          continue

        instancia_ids.append(indice)
        token_ids.append(tokenizer.convert_tokens_to_ids(tokens[i]))
        tokens_l.append(tokens[i])

        if inicio_pregunta:
          puntuacion_iniciales.append(1) #('"¿"')
          inicio_pregunta = False
        else:
          puntuacion_iniciales.append(0) #("")

        if i != len(tokens) - 1:

          if tokens[i+1] == "?":
            puntuacion_finales.append(3) #('"?"')
          elif tokens[i+1] == ".":
            puntuacion_finales.append(1) #('"."')
          elif tokens[i+1] == ",":
            puntuacion_finales.append(2) #('","')
          else:
            puntuacion_finales.append(0) #("")

        else:
          puntuacion_finales.append(0) #("")

        if palabra.islower():
          capitalizaciones.append(0)

        elif palabra.istitle():
          capitalizaciones.append(1)

        elif palabra.isupper():
          capitalizaciones.append(3)

        else:
          capitalizaciones.append(2)


  etiquetas = np.column_stack([instancia_ids, token_ids, tokens_l, puntuacion_iniciales, puntuacion_finales, capitalizaciones])
  return etiquetas

In [ ]:
def convertir_epub_a_csv(archivo_epub='libro.epub'):
  # Cargar el libro
  book = ebooklib.epub.read_epub(archivo_epub)

  # Lista donde se guardarán los párrafos
  parrafos = []

  # Recorremos los ítems del libro
  for item in book.get_items():
      if item.get_type() == ebooklib.ITEM_DOCUMENT:
          # Parseamos el contenido HTML
          soup = BeautifulSoup(item.get_body_content(), 'html.parser')
          # Extraemos los párrafos
          for p in soup.find_all('p'):
            #print("p:",p, 'tipo: ', type(p))
            texto = p.get_text().strip()
            #print("TEXTO:",texto, ' tipo: ', type(texto))
            palabras = texto.split()
            #print("PALABRAS:",palabras, ' tipo: ', type(palabras))
            if len(palabras) < 20 or len(palabras) > 100:  # descartamos párrafos cortos
                continue
            if texto:
                parrafos.append(texto)

  df = pd.DataFrame({'parrafo': parrafos})
  df.to_csv("libro_parrafos.csv", index=False, encoding="utf-8")

  print(f"Se extrajeron {len(parrafos)} párrafos y se guardaron en 'libro_parrafos.csv'.")

In [ ]:
def convertir_pdf_a_csv(pdf_path):
    """
    Convierte un archivo PDF en un CSV con una columna 'parrafo'.
    Divide el texto en los tipos de oraciónes que queremos apra entrenar
    """

    # Extraer texto del PDF
    texto_total = ""
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            texto = pagina.extract_text()
            if texto:
                texto_total += " " + texto

    # Limpieza básica
    texto_total = re.sub(r"-\n", "", texto_total)   # une palabras cortadas
    texto_total = re.sub(r"\s+", " ", texto_total)  # espacios normales
    texto_total = texto_total.strip()

    # Dividir el texto en oraciones por ".", "?", "!"
    # Pero conservamos los signos al final
    oraciones = re.split(r'(?<=[\.\?\!])\s+', texto_total)
    oraciones = [o.strip() for o in oraciones if len(o.strip()) > 2]

    #Agrupar cada 1–3 oraciones en un bloque
    bloques = []
    buffer = []
    for o in oraciones:
        buffer.append(o)
        # si ya hay 3 oraciones, o la actual termina en "?" o "!"
        if len(buffer) >= 3 or re.search(r'[?!]$', o):
            bloques.append(" ".join(buffer))
            buffer = []

    # lo que quede
    if buffer:
        bloques.append(" ".join(buffer))

    # Filtrar solo las que contienen puntuación real
    bloques = [b.strip() for b in bloques if re.search(r'[.,¿?!]|[a-zA-ZáéíóúÁÉÍÓÚñ]$', b)]

    df = pd.DataFrame(bloques, columns=['parrafo'])
    df.to_csv('libro_parrafos.csv', index=False, encoding='utf-8-sig')
    return df

#_______________________________________________________________________________________________
# VERSIÓN 2 DE PDF-> CSV

def convertir_pdf_a_csv2(pdf_path):
    """
    Convierte un PDF en CSV pero corta los textos de forma aleatoria:
    - A mitad de una oración
    - Cada 1 oración
    - Cada 2 oraciones
    - Siempre corta al final de una pregunta
    Además filtra párrafos con muchos números.
    """

    # -------- Extraer texto --------
    texto_total = ""
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            txt = pagina.extract_text()
            if txt:
                texto_total += " " + txt

    texto_total = re.sub(r"-\n", "", texto_total)
    texto_total = re.sub(r"\s+", " ", texto_total).strip()

    # -------- Dividir en oraciones --------
    oraciones = re.split(r'(?<=[\.\?\!])\s+', texto_total)
    oraciones = [o.strip() for o in oraciones if len(o.strip()) > 2]

    bloques = []
    i = 0
    while i < len(oraciones):

        # si aparece una pregunta → cortar ahí
        if "¿" in oraciones[i] or "?" in oraciones[i]:
            bloques.append(oraciones[i])
            i += 1
            continue

        # decisión aleatoria
        modo = np.random.choice(["mitad", "1", "2"], p=[0.2, 0.4, 0.4])

        if modo == "mitad":
            # cortar a la mitad de la oración
            oracion = oraciones[i]
            palabras = oracion.split()
            mitad = max(3, len(palabras) // 2)
            bloque = " ".join(palabras[:mitad])  # mitad truncada
            bloques.append(bloque)
            i += 1

        elif modo == "1":
            bloques.append(oraciones[i])
            i += 1

        else:  # modo = "2"
            if i + 1 < len(oraciones):
                bloques.append(oraciones[i] + " " + oraciones[i+1])
                i += 2
            else:
                bloques.append(oraciones[i])
                i += 1

    # -------- Filtrar párrafos con demasiados números --------
    def demasiado_numeros(texto):
        nums = sum(c.isdigit() for c in texto)
        return nums / max(len(texto), 1) > 0.15 or bool(re.search(r"\b\d+(\.\d+)*\b", texto))

    bloques = [b for b in bloques if not demasiado_numeros(b)]

    # -------- Balancear preguntas --------
    preg = [b for b in bloques if "¿" in b or "?" in b]
    nopreg = [b for b in bloques if "¿" not in b and "?" not in b]

    # eliminar 75% de nopreg
    cant = int(len(nopreg) * 0.25)
    nopreg_reducidas = list(np.random.choice(nopreg, cant, replace=False)) if cant > 0 else []

    # combinar todo
    bloques_final = preg + nopreg_reducidas

    # mezclar aleatoriamente
    np.random.shuffle(bloques_final)

    df = pd.DataFrame(bloques_final, columns=['parrafo'])
    df.to_csv('libro_parrafos.csv', index=False, encoding='utf-8-sig')

    return df



In [ ]:
def crearDataSet(libro,i,pdf=False):
  if pdf:
    convertir_pdf_a_csv2(libro)
  else:
    convertir_epub_a_csv(libro)

  df = pd.read_csv('libro_parrafos.csv')

  parrafos = pd.DataFrame(columns=['default', 'limpio'])
  parrafos['limpio'] = df['parrafo'].apply(normalize_text_X)
  parrafos['default'] = df['parrafo'].apply(normalize_text_y)

  columnas = ['instancia_id', 'token_id', 'token', 'punt_inicial', 'punt_final', 'capitalización']
  datos = pd.DataFrame(columns=columnas)

  for p in parrafos['default']:
    etiquetas = transformar_etiqueta([p], i)
    etiquetas = pd.DataFrame(etiquetas, columns=columnas)
    datos = pd.concat([datos, etiquetas], ignore_index=True)
    i += 1

  print('Data set creado de tamaño: ', datos.shape)

  numeric_cols = ['instancia_id', 'token_id', 'punt_inicial', 'punt_final', 'capitalización']
  for col in numeric_cols:
    datos[col] = pd.to_numeric(datos[col], errors='coerce')

  return parrafos, datos, i

In [ ]:
def agregar_embeddings(df_X):
  token_ids = df_X['token_id'].tolist()
  embeddings_list = []
  for token_id  in token_ids:
      if token_id is None or token_id == tokenizer.unk_token_id:
        token_id = tokenizer.unk_token_id
      # Detach the tensor before converting to list
      embedding = model.embeddings.word_embeddings.weight[token_id].detach().tolist()
      embeddings_list.append(embedding)
  df_X['embeddings'] = embeddings_list
  # The embeddings are already lists of floats, no need to convert to int
  # df_X['embeddings'] = df_X['embeddings'].apply(lambda x: [int(i) for i in x])

  return df_X

In [ ]:
def trasformar_df_dfPyTorch(datos_X, datos_Y):
  # Convert the list of lists in the 'embeddings' column to a NumPy array of floats
  embeddings_array = np.array(datos_X['embeddings'].tolist(), dtype=np.float32)
  X = torch.tensor(embeddings_array, dtype=torch.float32)
  Y = torch.tensor(datos_Y[['punt_inicial', 'punt_final', 'capitalización']].values, dtype=torch.float32)
  dataSetPT = TensorDataset(X, Y)

  return dataSetPT

### Puntuacion inicial:
*   "": 0
*   "¿": 1

### Puntuacion final:
*   "": 0
*   ".": 1
*   ",": 2
*   "?": 3



### Dataset basico con un libro de Harry Potter

Busca libro en formato epub de mi github

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'

In [ ]:
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'

In [ ]:
path = path + libro1

In [ ]:
!wget -O libro1.epub $path

In [ ]:
parrafos, dataSet = crearDataSet('libro1.epub')

In [ ]:
datos_X = dataSet[['instancia_id', 'token_id', 'token']]
datos_Y = dataSet[['punt_inicial', 'punt_final', 'capitalización']]

In [ ]:
datos_X_ext = agregar_embeddings(datos_X)

In [ ]:
dataSetPT = trasformar_df_dfPyTorch(datos_X, datos_Y)

In [ ]:
X0, Y0 = dataSetPT[0]
print("X[0]:", X0)
print("Y[0]:", Y0)

### Dataser definitivo con muchos libros pdf

In [ ]:
carpeta = "libros_pdf"
#todos_parrafos = pd.DataFrame(columns=['default', 'limpio'])
#todos_datos = pd.DataFrame(columns=['instancia_id', 'token_id', 'token', 'punt_inicial', 'punt_final', 'capitalización'])

todos_parrafos = pd.read_csv("/content/drive/MyDrive/colab/dataset_parrafos_total3.csv")
todos_datos    = pd.read_csv("/content/drive/MyDrive/colab/dataset_datos_total3.csv")

i = len(todos_parrafos) #0
l = 129 #0
lista_de_libros = os.listdir(carpeta)
libros_restantes = lista_de_libros[l:]
for nombre in libros_restantes: #os.listdir(carpeta):
    ruta = os.path.join(carpeta, nombre)
    try:
      if nombre.lower().endswith(".pdf"):
        parrafos, datos, i = crearDataSet(ruta,i, pdf=True)
      elif nombre.lower().endswith(".epub"):
        parrafos, datos = crearDataSet(ruta, pdf=False)
      else:
        print(f"⚠️ Saltando {nombre} (no es PDF ni EPUB)")
        continue
    except Exception as e:
      print(f"❌ Error procesando {nombre}: {e}")
      continue
    todos_parrafos = pd.concat([todos_parrafos, parrafos], ignore_index=True)
    todos_datos = pd.concat([todos_datos, datos], ignore_index=True)
    print(f"✅ Procesado el libro {l}: {nombre} ({len(parrafos)} párrafos, {len(datos)} tokens)")
    print(f"Cantidad de parrafos totales: {len(todos_parrafos)}")
    print(f"Cantidad de tokens totales: {len(todos_datos)}")
    l += 1
    todos_parrafos.to_csv("/content/drive/MyDrive/colab/dataset_parrafos_total3.csv", index=False)
    todos_datos.to_csv("/content/drive/MyDrive/colab/dataset_datos_total3.csv", index=False)
    print(f"Se guardó automáticamente en Drive")

print(f"\n💾 Dataset total creado: {len(todos_parrafos)} párrafos y {len(todos_datos)} tokens.")


In [ ]:
#Mostramos el print de la celda anterior de como va creando el dataset
"""
Data set creado de tamaño:  (6960, 6)
✅ Procesado el libro 129: El juego de la vida y cómo jugarlo autor Florence Scovel Shinn.pdf (234 párrafos, 6960 tokens)
Cantidad de parrafos totales: 149044
Cantidad de tokens totales: 4199231
Se guardó automáticamente en Drive
Data set creado de tamaño:  (20676, 6)
✅ Procesado el libro 130: El libro de la selva, Rudyard Kipling.pdf (968 párrafos, 20676 tokens)
Cantidad de parrafos totales: 150012
Cantidad de tokens totales: 4219907
Se guardó automáticamente en Drive
Data set creado de tamaño:  (34832, 6)
✅ Procesado el libro 131: 1984 Autor George Orwell.pdf (1436 párrafos, 34832 tokens)
Cantidad de parrafos totales: 151448
Cantidad de tokens totales: 4254739
Se guardó automáticamente en Drive
Data set creado de tamaño:  (25964, 6)
✅ Procesado el libro 132: 2  La isla del tesoro autor Robert Louise Stenvenson.pdf (968 párrafos, 25964 tokens)
Cantidad de parrafos totales: 152416
Cantidad de tokens totales: 4280703
Se guardó automáticamente en Drive
Data set creado de tamaño:  (15679, 6)
✅ Procesado el libro 133: 11. El castillo de Otranto, Horace Walpole.pdf (688 párrafos, 15679 tokens)
Cantidad de parrafos totales: 153104
Cantidad de tokens totales: 4296382
Se guardó automáticamente en Drive
Data set creado de tamaño:  (15673, 6)
✅ Procesado el libro 134: 14. The Turn of the Screw, Henry James.pdf (543 párrafos, 15673 tokens)
Cantidad de parrafos totales: 153647
Cantidad de tokens totales: 4312055
Se guardó automáticamente en Drive
Data set creado de tamaño:  (16910, 6)
✅ Procesado el libro 135: 27 Las penas del joven Werther autor Johann Wolfgang von Goethe.pdf (557 párrafos, 16910 tokens)
Cantidad de parrafos totales: 154204
Cantidad de tokens totales: 4328965
Se guardó automáticamente en Drive
❌ Error procesando 126 - El inversor inteligente - Warren Buffett.pdf: Unexpected EOF
Data set creado de tamaño:  (103824, 6)
✅ Procesado el libro 136: 06. Melmoth el errabundo Autor Charles Robert Maturin.pdf (2574 párrafos, 103824 tokens)
Cantidad de parrafos totales: 156778
Cantidad de tokens totales: 4432789
Se guardó automáticamente en Drive
Data set creado de tamaño:  (35110, 6)
✅ Procesado el libro 137: 127-Coaching para Dummies - Jeni Mumford.pdf (1491 párrafos, 35110 tokens)
Cantidad de parrafos totales: 158269
Cantidad de tokens totales: 4467899
Se guardó automáticamente en Drive
Data set creado de tamaño:  (3754, 6)
✅ Procesado el libro 138: La confianza en uno mismo autor Ralph Waldo Emerson.pdf (115 párrafos, 3754 tokens)
Cantidad de parrafos totales: 158384
Cantidad de tokens totales: 4471653
Se guardó automáticamente en Drive
Data set creado de tamaño:  (70477, 6)
✅ Procesado el libro 139: La República Autor Platón.pdf (2377 párrafos, 70477 tokens)
Cantidad de parrafos totales: 160761
Cantidad de tokens totales: 4542130
Se guardó automáticamente en Drive
Data set creado de tamaño:  (41475, 6)
✅ Procesado el libro 140: 05. Los siete locos Autor Roberto Arlt.pdf (1845 párrafos, 41475 tokens)
Cantidad de parrafos totales: 162606
Cantidad de tokens totales: 4583605
Se guardó automáticamente en Drive
Data set creado de tamaño:  (52870, 6)
✅ Procesado el libro 141: Rayuela Autor Julio Cortazar.pdf (2012 párrafos, 52870 tokens)
Cantidad de parrafos totales: 164618
Cantidad de tokens totales: 4636475
Se guardó automáticamente en Drive
Data set creado de tamaño:  (8659, 6)
✅ Procesado el libro 142: 31 La historia del doctor Dolittle autor Hugh Lofting.pdf (386 párrafos, 8659 tokens)
Cantidad de parrafos totales: 165004
Cantidad de tokens totales: 4645134
Se guardó automáticamente en Drive
Data set creado de tamaño:  (20017, 6)
✅ Procesado el libro 143: las-aventuras-de-tom-sawyer-mark-twain.pdf (1082 párrafos, 20017 tokens)
Cantidad de parrafos totales: 166086
Cantidad de tokens totales: 4665151
Se guardó automáticamente en Drive
Data set creado de tamaño:  (67334, 6)
✅ Procesado el libro 144: PODER SIN LIMITES - ANTHONY ROBBINS 1.pdf (1880 párrafos, 67334 tokens)
Cantidad de parrafos totales: 167966
Cantidad de tokens totales: 4732485
Se guardó automáticamente en Drive
Data set creado de tamaño:  (27173, 6)
✅ Procesado el libro 145: el-talon-de-hierro-jack-london.pdf (923 párrafos, 27173 tokens)
Cantidad de parrafos totales: 168889
Cantidad de tokens totales: 4759658
Se guardó automáticamente en Drive
Data set creado de tamaño:  (3655, 6)
✅ Procesado el libro 146: EMPRENDEDOR - ANTHONY ROBBINS.pdf (162 párrafos, 3655 tokens)
Cantidad de parrafos totales: 169051
Cantidad de tokens totales: 4763313
Se guardó automáticamente en Drive
Data set creado de tamaño:  (11676, 6)
✅ Procesado el libro 147: El Kybalion Autor Tres Iniciados.pdf (266 párrafos, 11676 tokens)
Cantidad de parrafos totales: 169317
Cantidad de tokens totales: 4774989
Se guardó automáticamente en Drive
Data set creado de tamaño:  (93789, 6)
✅ Procesado el libro 148: 10 Mujeres enamoradas autor D H Lawrence.pdf (4221 párrafos, 93789 tokens)
Cantidad de parrafos totales: 173538
Cantidad de tokens totales: 4868778
Se guardó automáticamente en Drive
Data set creado de tamaño:  (24138, 6)
✅ Procesado el libro 149: Un Mundo Feliz Autor Aldous Huxley.pdf (1148 párrafos, 24138 tokens)
Cantidad de parrafos totales: 174686
Cantidad de tokens totales: 4892916
Se guardó automáticamente en Drive
Data set creado de tamaño:  (68461, 6)
✅ Procesado el libro 150: 15 Emma autor Jane Austen.pdf (2354 párrafos, 68461 tokens)
Cantidad de parrafos totales: 177040
Cantidad de tokens totales: 4961377
Se guardó automáticamente en Drive
Data set creado de tamaño:  (24318, 6)
✅ Procesado el libro 151: un-mundo-feliz-aldous-huxley.pdf (1140 párrafos, 24318 tokens)
Cantidad de parrafos totales: 178180
Cantidad de tokens totales: 4985695
Se guardó automáticamente en Drive
Data set creado de tamaño:  (33096, 6)
✅ Procesado el libro 152: 18 Mujercitas autor Louisa May Alcott.pdf (1142 párrafos, 33096 tokens)
Cantidad de parrafos totales: 179322
Cantidad de tokens totales: 5018791
Se guardó automáticamente en Drive
Data set creado de tamaño:  (35748, 6)
✅ Procesado el libro 153: El Amor en Tiempo de Cólera Autor Gabriel García Márquez.pdf (744 párrafos, 35748 tokens)
Cantidad de parrafos totales: 180066
Cantidad de tokens totales: 5054539
Se guardó automáticamente en Drive
Data set creado de tamaño:  (7531, 6)
✅ Procesado el libro 154: Los Cuatro Acuerdos Autor Miguel Ruiz.pdf (322 párrafos, 7531 tokens)
Cantidad de parrafos totales: 180388
Cantidad de tokens totales: 5062070
Se guardó automáticamente en Drive
Data set creado de tamaño:  (9046, 6)
✅ Procesado el libro 155: 024-El vendedor más grande del mundo- Og Mandin.pdf (364 párrafos, 9046 tokens)
Cantidad de parrafos totales: 180752
Cantidad de tokens totales: 5071116
Se guardó automáticamente en Drive
Data set creado de tamaño:  (49259, 6)
✅ Procesado el libro 156: VENTAS PARA DUMMIES - TOM HOPKINS.pdf (1538 párrafos, 49259 tokens)
Cantidad de parrafos totales: 182290
Cantidad de tokens totales: 5120375
Se guardó automáticamente en Drive
Data set creado de tamaño:  (53762, 6)
✅ Procesado el libro 157: Perdiendo la virginidad- Richard Branson.pdf (1810 párrafos, 53762 tokens)
Cantidad de parrafos totales: 184100
Cantidad de tokens totales: 5174137
Se guardó automáticamente en Drive
Data set creado de tamaño:  (37750, 6)
✅ Procesado el libro 158: El Diario de Ana Frank Autor Ana Frank.pdf (1329 párrafos, 37750 tokens)
Cantidad de parrafos totales: 185429
Cantidad de tokens totales: 5211887
Se guardó automáticamente en Drive
Data set creado de tamaño:  (23577, 6)
✅ Procesado el libro 159: 13 Peter Pan y Wendy autor James Matthew Barrie.pdf (989 párrafos, 23577 tokens)
Cantidad de parrafos totales: 186418
Cantidad de tokens totales: 5235464
Se guardó automáticamente en Drive
Data set creado de tamaño:  (41857, 6)
✅ Procesado el libro 160: 21 El rey del mar autor Emilio Salgari.pdf (1866 párrafos, 41857 tokens)
Cantidad de parrafos totales: 188284
Cantidad de tokens totales: 5277321
Se guardó automáticamente en Drive
Data set creado de tamaño:  (9683, 6)
✅ Procesado el libro 161: Bhagavad Gita autor Viasa (1).pdf (255 párrafos, 9683 tokens)
Cantidad de parrafos totales: 188539
Cantidad de tokens totales: 5287004
Se guardó automáticamente en Drive
Data set creado de tamaño:  (44359, 6)
✅ Procesado el libro 162: 01. Dona Barbara  Autor Romulo Gallegos.pdf (1384 párrafos, 44359 tokens)
Cantidad de parrafos totales: 189923
Cantidad de tokens totales: 5331363
Se guardó automáticamente en Drive
Data set creado de tamaño:  (27560, 6)
✅ Procesado el libro 163: 20 Persuasión autor Jane Austen.pdf (994 párrafos, 27560 tokens)
Cantidad de parrafos totales: 190917
Cantidad de tokens totales: 5358923
WARNING:pdfplumber.pdf:[WARNING] Metadata key "CreationDate" could not be parsed due to exception: maximum recursion depth exceeded
WARNING:pdfplumber.pdf:[WARNING] Metadata key "Creator" could not be parsed due to exception: maximum recursion depth exceeded
Se guardó automáticamente en Drive
Data set creado de tamaño:  (2278, 6)
✅ Procesado el libro 164: Dhammapada Autor Buda.pdf (89 párrafos, 2278 tokens)
Cantidad de parrafos totales: 191006
Cantidad de tokens totales: 5361201
Se guardó automáticamente en Drive
Data set creado de tamaño:  (16814, 6)
✅ Procesado el libro 165: 096-El Arte De Cautivar - Guy Kawasaki.pdf (601 párrafos, 16814 tokens)
Cantidad de parrafos totales: 191607
Cantidad de tokens totales: 5378015
Se guardó automáticamente en Drive
Data set creado de tamaño:  (52854, 6)
✅ Procesado el libro 166: 10. El Monje, Matthew Lewis.pdf (2101 párrafos, 52854 tokens)
Cantidad de parrafos totales: 193708
Cantidad de tokens totales: 5430869
Se guardó automáticamente en Drive
Data set creado de tamaño:  (21659, 6)
✅ Procesado el libro 167: el-proceso-franz-kafka.pdf (927 párrafos, 21659 tokens)
Cantidad de parrafos totales: 194635
Cantidad de tokens totales: 5452528
Se guardó automáticamente en Drive
Data set creado de tamaño:  (28822, 6)
✅ Procesado el libro 168: 30 Una Habitación con Vistas autor E. M. Foster.pdf (1487 párrafos, 28822 tokens)
Cantidad de parrafos totales: 196122
Cantidad de tokens totales: 5481350
Se guardó automáticamente en Drive
Data set creado de tamaño:  (22414, 6)
✅ Procesado el libro 169: 20907-el-proceso-franz-kafka.pdf (926 párrafos, 22414 tokens)
Cantidad de parrafos totales: 197048
Cantidad de tokens totales: 5503764
Se guardó automáticamente en Drive
Data set creado de tamaño:  (45459, 6)
✅ Procesado el libro 170: 32 Las alegres aventuras autor Robin Hood Howard Pyle.pdf (1414 párrafos, 45459 tokens)
Cantidad de parrafos totales: 198462
Cantidad de tokens totales: 5549223
Se guardó automáticamente en Drive
Data set creado de tamaño:  (43618, 6)
✅ Procesado el libro 171: 150 - Pensar Rápido, Pensar Despacio - Daniel K.pdf (1318 párrafos, 43618 tokens)
Cantidad de parrafos totales: 199780
Cantidad de tokens totales: 5592841
Se guardó automáticamente en Drive
Data set creado de tamaño:  (11921, 6)
✅ Procesado el libro 172: Meditaciones Autor Marco Aurelio.pdf (458 párrafos, 11921 tokens)
Cantidad de parrafos totales: 200238
Cantidad de tokens totales: 5604762
Se guardó automáticamente en Drive
Data set creado de tamaño:  (26061, 6)
✅ Procesado el libro 173: TUS ZONAS ERRONEAS - WAYNE W. DYER - 243 PAGINA.pdf (886 párrafos, 26061 tokens)
Cantidad de parrafos totales: 201124
Cantidad de tokens totales: 5630823
Se guardó automáticamente en Drive
Data set creado de tamaño:  (22507, 6)
✅ Procesado el libro 174: PIENSE Y HAGASE RICO - NAPOLEON HILL - 174 PAGI.pdf (729 párrafos, 22507 tokens)
Cantidad de parrafos totales: 201853
Cantidad de tokens totales: 5653330
Se guardó automáticamente en Drive
Data set creado de tamaño:  (10117, 6)
✅ Procesado el libro 175: El Kybalion Autor Tres Iniciados (1).pdf (262 párrafos, 10117 tokens)
Cantidad de parrafos totales: 202115
Cantidad de tokens totales: 5663447
Se guardó automáticamente en Drive
Data set creado de tamaño:  (27098, 6)
✅ Procesado el libro 176: 1 La isla del tesoro autor Robert Louise Stenvenson.pdf (963 párrafos, 27098 tokens)
Cantidad de parrafos totales: 203078
Cantidad de tokens totales: 5690545
Se guardó automáticamente en Drive
Data set creado de tamaño:  (33724, 6)
✅ Procesado el libro 177: La rueda de la vida- Elisabeth Kübler-Ross.pdf (1230 párrafos, 33724 tokens)
Cantidad de parrafos totales: 204308
Cantidad de tokens totales: 5724269
Se guardó automáticamente en Drive
Data set creado de tamaño:  (27363, 6)
✅ Procesado el libro 178: mujercitas-louisa-may-alcott.pdf (1014 párrafos, 27363 tokens)
Cantidad de parrafos totales: 205322
Cantidad de tokens totales: 5751632
Se guardó automáticamente en Drive
Data set creado de tamaño:  (43309, 6)
✅ Procesado el libro 179: Así habló Zaratustra Autor Friedrich Wilhelm Nietzsche (2).pdf (1542 párrafos, 43309 tokens)
Cantidad de parrafos totales: 206864
Cantidad de tokens totales: 5794941
Se guardó automáticamente en Drive
Data set creado de tamaño:  (3541, 6)
✅ Procesado el libro 180: 40 El fantasma de Canterville autor Oscar Wilde.pdf (96 párrafos, 3541 tokens)
Cantidad de parrafos totales: 206960
Cantidad de tokens totales: 5798482
Se guardó automáticamente en Drive
Data set creado de tamaño:  (22903, 6)
✅ Procesado el libro 181: EL ARTE DE CERRAR LA VENTA - BRIAN TRACY - 212 .pdf (636 párrafos, 22903 tokens)
Cantidad de parrafos totales: 207596
Cantidad de tokens totales: 5821385
Se guardó automáticamente en Drive
Data set creado de tamaño:  (46030, 6)
✅ Procesado el libro 182: 3 Madame Bovary autor Gustave Flaubert.pdf (1518 párrafos, 46030 tokens)
Cantidad de parrafos totales: 209114
Cantidad de tokens totales: 5867415
Se guardó automáticamente en Drive
Data set creado de tamaño:  (9389, 6)
✅ Procesado el libro 183: 068-Gopro, 7 pasos para convertirse en un profe.pdf (462 párrafos, 9389 tokens)
Cantidad de parrafos totales: 209576
Cantidad de tokens totales: 5876804
Se guardó automáticamente en Drive
Data set creado de tamaño:  (6463, 6)
✅ Procesado el libro 184: 08. Carmilla, Sheridan Le Fanu.pdf (292 párrafos, 6463 tokens)
Cantidad de parrafos totales: 209868
Cantidad de tokens totales: 5883267
Se guardó automáticamente en Drive
Data set creado de tamaño:  (18170, 6)
✅ Procesado el libro 185: Piense y Hágase Rico Autor Napoleon Hill.pdf (653 párrafos, 18170 tokens)
Cantidad de parrafos totales: 210521
Cantidad de tokens totales: 5901437
Se guardó automáticamente en Drive
Data set creado de tamaño:  (9548, 6)
✅ Procesado el libro 186: 160-Padre rico, padre pobre para jóvenes - Robe.pdf (389 párrafos, 9548 tokens)
Cantidad de parrafos totales: 210910
Cantidad de tokens totales: 5910985
Se guardó automáticamente en Drive
Data set creado de tamaño:  (34586, 6)
✅ Procesado el libro 187: Leviatán Autor Thomas Hobbes.pdf (622 párrafos, 34586 tokens)
Cantidad de parrafos totales: 211532
Cantidad de tokens totales: 5945571
Se guardó automáticamente en Drive
Data set creado de tamaño:  (20902, 6)
✅ Procesado el libro 188: El Lider Que No Tenia Cargo.pdf (994 párrafos, 20902 tokens)
Cantidad de parrafos totales: 212526
Cantidad de tokens totales: 5966473
Se guardó automáticamente en Drive
Data set creado de tamaño:  (12620, 6)
✅ Procesado el libro 189: El Alquimista Autor Paulo Coelho.pdf (598 párrafos, 12620 tokens)
Cantidad de parrafos totales: 213124
Cantidad de tokens totales: 5979093
Se guardó automáticamente en Drive
Data set creado de tamaño:  (72611, 6)
✅ Procesado el libro 190: 29 Grandes esperanzas autor Charles Dickens.pdf (2580 párrafos, 72611 tokens)
Cantidad de parrafos totales: 215704
Cantidad de tokens totales: 6051704
Se guardó automáticamente en Drive
Data set creado de tamaño:  (29041, 6)
✅ Procesado el libro 191: 090-Tus Zonas Erróneas - Wayne Dyer.pdf (918 párrafos, 29041 tokens)
Cantidad de parrafos totales: 216622
Cantidad de tokens totales: 6080745
Se guardó automáticamente en Drive
Data set creado de tamaño:  (45987, 6)
✅ Procesado el libro 192: 6 Madame Bovary autor Gustave Flaubert.pdf (1527 párrafos, 45987 tokens)
Cantidad de parrafos totales: 218149
Cantidad de tokens totales: 6126732
Se guardó automáticamente en Drive
Data set creado de tamaño:  (16046, 6)
✅ Procesado el libro 193: 020-El Hombre Mas Rico De Babilonia - George S..pdf (586 párrafos, 16046 tokens)
Cantidad de parrafos totales: 218735
Cantidad de tokens totales: 6142778
Se guardó automáticamente en Drive
Data set creado de tamaño:  (43398, 6)
✅ Procesado el libro 194: Michael Jordan- Maximo Jose Tobias.pdf (963 párrafos, 43398 tokens)
Cantidad de parrafos totales: 219698
Cantidad de tokens totales: 6186176
Se guardó automáticamente en Drive
Data set creado de tamaño:  (25212, 6)
✅ Procesado el libro 195: El color de la libertad - Nelson Mandela.pdf (687 párrafos, 25212 tokens)
Cantidad de parrafos totales: 220385
Cantidad de tokens totales: 6211388
Se guardó automáticamente en Drive
Data set creado de tamaño:  (4629, 6)
✅ Procesado el libro 196: 2 Los crímenes de la calle Morgue autor Edgar Allan Poe.pdf (152 párrafos, 4629 tokens)
Cantidad de parrafos totales: 220537
Cantidad de tokens totales: 6216017
Se guardó automáticamente en Drive
Data set creado de tamaño:  (43786, 6)
✅ Procesado el libro 197: Así habló Zaratustra Autor Friedrich Wilhelm Nietzsche (1).pdf (1546 párrafos, 43786 tokens)
Cantidad de parrafos totales: 222083
Cantidad de tokens totales: 6259803
Se guardó automáticamente en Drive
Data set creado de tamaño:  (23556, 6)
✅ Procesado el libro 198: pobres-gentes-fyodor-dostoyevsky (1).pdf (862 párrafos, 23556 tokens)
Cantidad de parrafos totales: 222945
Cantidad de tokens totales: 6283359
Se guardó automáticamente en Drive
Data set creado de tamaño:  (16519, 6)
✅ Procesado el libro 199: EL HOMBRE MAS RICO DE BABILONIA - GEORGE S. CLA.pdf (587 párrafos, 16519 tokens)
Cantidad de parrafos totales: 223532
Cantidad de tokens totales: 6299878
Se guardó automáticamente en Drive
Data set creado de tamaño:  (26518, 6)
✅ Procesado el libro 200: El Retrato de Dorian Gray Autor Oscar Wilde.pdf (1483 párrafos, 26518 tokens)
Cantidad de parrafos totales: 225015
Cantidad de tokens totales: 6326396
Se guardó automáticamente en Drive
Data set creado de tamaño:  (42223, 6)
✅ Procesado el libro 201: 077-MBA personal - Josh Kaufman.pdf (1383 párrafos, 42223 tokens)
Cantidad de parrafos totales: 226398
Cantidad de tokens totales: 6368619
Se guardó automáticamente en Drive
Data set creado de tamaño:  (17380, 6)
✅ Procesado el libro 202: 29 Azabache autor Anna Sewell.pdf (603 párrafos, 17380 tokens)
Cantidad de parrafos totales: 227001
Cantidad de tokens totales: 6385999
Se guardó automáticamente en Drive
Data set creado de tamaño:  (26727, 6)
✅ Procesado el libro 203: La Divina Comedia Autor Dante Alighieri.pdf (508 párrafos, 26727 tokens)
Cantidad de parrafos totales: 227509
Cantidad de tokens totales: 6412726
Se guardó automáticamente en Drive
Data set creado de tamaño:  (5750, 6)
✅ Procesado el libro 204: El Principito Autor Antoine de Saint-Exupéry.pdf (356 párrafos, 5750 tokens)
Cantidad de parrafos totales: 227865
Cantidad de tokens totales: 6418476
Se guardó automáticamente en Drive
Data set creado de tamaño:  (44002, 6)
✅ Procesado el libro 205: 047-Einstein, Su Vida y su Universo - Walter Is.pdf (1264 párrafos, 44002 tokens)
Cantidad de parrafos totales: 229129
Cantidad de tokens totales: 6462478
Se guardó automáticamente en Drive
Data set creado de tamaño:  (23642, 6)
✅ Procesado el libro 206: pobres-gentes-fyodor-dostoyevsky.pdf (858 párrafos, 23642 tokens)
Cantidad de parrafos totales: 229987
Cantidad de tokens totales: 6486120
Se guardó automáticamente en Drive
Data set creado de tamaño:  (23064, 6)
✅ Procesado el libro 207: FUERA DE SERIE - PORQUE UNAS PERSONAS TIENEN EX.pdf (842 párrafos, 23064 tokens)
Cantidad de parrafos totales: 230829
Cantidad de tokens totales: 6509184
Se guardó automáticamente en Drive
Data set creado de tamaño:  (40646, 6)
✅ Procesado el libro 208: 02. María Autor Jorge Isaacs.pdf (1598 párrafos, 40646 tokens)
Cantidad de parrafos totales: 232427
Cantidad de tokens totales: 6549830
Se guardó automáticamente en Drive
Data set creado de tamaño:  (30156, 6)
✅ Procesado el libro 209: El codigo Federer- Stefano Semeraro y Ana Ciura.pdf (916 párrafos, 30156 tokens)
Cantidad de parrafos totales: 233343
Cantidad de tokens totales: 6579986
Se guardó automáticamente en Drive
Data set creado de tamaño:  (51266, 6)
✅ Procesado el libro 210: Drácula Autor Bram Stoker.pdf (1757 párrafos, 51266 tokens)
Cantidad de parrafos totales: 235100
Cantidad de tokens totales: 6631252
Se guardó automáticamente en Drive
Data set creado de tamaño:  (17727, 6)
✅ Procesado el libro 211: 117-Incrementa Tu IQ Financiero - Robert T Kiyo.pdf (537 párrafos, 17727 tokens)
Cantidad de parrafos totales: 235637
Cantidad de tokens totales: 6648979
Se guardó automáticamente en Drive
Data set creado de tamaño:  (25681, 6)
✅ Procesado el libro 212: 064-El Toque de Midas - Donald Trump y Robert K.pdf (945 párrafos, 25681 tokens)
Cantidad de parrafos totales: 236582
Cantidad de tokens totales: 6674660
Se guardó automáticamente en Drive
Data set creado de tamaño:  (14059, 6)
✅ Procesado el libro 213: 11 Las aventuras de Alicia en el país de las maravillas autor  Lewis Carroll.pdf (519 párrafos, 14059 tokens)
Cantidad de parrafos totales: 237101
Cantidad de tokens totales: 6688719
Se guardó automáticamente en Drive
Data set creado de tamaño:  (23249, 6)
✅ Procesado el libro 214: 03. Frankenstein, Mary Shelley.pdf (689 párrafos, 23249 tokens)
Cantidad de parrafos totales: 237790
Cantidad de tokens totales: 6711968
Se guardó automáticamente en Drive
Data set creado de tamaño:  (60661, 6)
✅ Procesado el libro 215: 33 Tess, D’Urberville autor Thomas Hardy.pdf (2036 párrafos, 60661 tokens)
Cantidad de parrafos totales: 239826
Cantidad de tokens totales: 6772629
Se guardó automáticamente en Drive
Data set creado de tamaño:  (82390, 6)
✅ Procesado el libro 216: 049-El Lobo de Wall Street - Jordan Belfort.pdf (3809 párrafos, 82390 tokens)
Cantidad de parrafos totales: 243635
Cantidad de tokens totales: 6855019
Se guardó automáticamente en Drive

💾 Dataset total creado: 243635 párrafos y 6855019 tokens.
"""